In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 

set_to_release = 'd3c98960-12e9-4a55-8971-151eaf613a6f'
store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store={}, item_uuids=[])
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)

115
115
1.5021759311358134


In [3]:
# Release the items
action = True
change_status = 'released'

counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'access_key', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)

ALREADY RELEASED/TO PROJECT workflow_run_sbg released 98627405-e039-4bab-98eb-ed104d5f77dc
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 0028fe18-689d-4db7-b977-77a155eb5624
ALREADY RELEASED/TO PROJECT workflow_run_sbg released e78c3a8d-dc7e-4c87-860e-8a1a98f2cea4
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 42cc4c5a-d473-480a-9791-76c86fe1ec50
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 2b71edce-4e0e-4a5c-abd9-940a3b619de2
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 63fd0bb7-88ee-48dc-bfbc-765965397478
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 4ce60c56-f112-44f3-97a4-24d6532a70ef
ALREADY RELEASED/TO PROJECT workflow_run_sbg released d5dc0220-a708-4e3f-962e-0e2aec76990b
ALREADY RELEASED/TO PROJECT workflow_run_sbg released b66ef6c2-89a6-4360-91ac-30271d55968f
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 61759391-0826-42e3-b29e-0486ad885675
ALREADY RELEASED/TO PROJECT workflow_run_sbg released 6ff18bd8-cab2-46aa-a454-ca379261407f